LSTM trained on gridded forcings for each station

In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../..')
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from datetime import datetime, timedelta
from sklearn import preprocessing
import netCDF4 as nc
import torch
from torch import nn
from torch.utils.tensorboard import SummaryWriter
from src import load_data, evaluate, datasets, utils
import torch.autograd as autograd

time_stamp = datetime.now().strftime('%Y%m%d-%H%M%S')
time_stamp

'20190904-221524'

In [2]:
USE_CUDA = False
if torch.cuda.is_available():
    print('CUDA Available')
    USE_CUDA = True
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
device = torch.device('cuda:0' if USE_CUDA else 'cpu')
num_devices = torch.cuda.device_count() if USE_CUDA else 0
print('cuda devices: {}'.format(list(torch.cuda.get_device_name(i) for i in range(num_devices))))
torch.manual_seed(0)
np.random.seed(0)

writer = SummaryWriter()

CUDA Available
cuda devices: ['Tesla V100-SXM2-16GB']


In [3]:
rdrs_vars = [4,5]
agg = None
seq_len = 5*24
seq_steps = 1
validation_fraction = 0.1
learning_rate = 2e-3
patience = 100
min_improvement = 0.01
batch_size = 4
H = 20
lstm_layers = 1
epochs = 400
train_start = datetime.strptime('2010-01-01', '%Y-%m-%d') + timedelta(hours=seq_len * seq_steps)
train_end = '2012-12-31'
test_start = '2013-01-01'
test_end = '2014-12-31'

In [4]:
# For each station, read which grid cells belong to its subwatershed
station_cell_mapping = load_data.get_station_cell_mapping()

In [5]:
def get_datasets(station):
    train_dataset = datasets.RdrsDataset(rdrs_vars, seq_len, seq_steps, train_start, train_end, include_months=False, aggregate_daily=agg, station=station)
    test_dataset = datasets.RdrsDataset(rdrs_vars, seq_len, seq_steps, test_start, test_end, include_months=False, aggregate_daily=agg,
                                        conv_scalers=train_dataset.conv_scalers, fc_scalers=train_dataset.fc_scalers, station=station)

    val_indices = np.random.choice(len(train_dataset), size=int(validation_fraction * len(train_dataset)), replace=False)
    train_indices = list(i for i in range(len(train_dataset)) if i not in val_indices)
    train_sampler = torch.utils.data.SubsetRandomSampler(train_indices)
    val_sampler = torch.utils.data.SubsetRandomSampler(val_indices)
    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size, sampler=train_sampler, pin_memory=True, drop_last=False)
    val_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size, sampler=val_sampler, pin_memory=True, drop_last=False)
    test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size, shuffle=False, pin_memory=True, drop_last=False)
    
    return train_dataset, test_dataset, train_dataloader, val_dataloader, test_dataloader

In [6]:
class LSTMRegression(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, batch_size):
        super(LSTMRegression, self).__init__()
        self.batch_size = batch_size
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers)
        self.linear = nn.Linear(hidden_dim, 1)
        self.init_hidden(batch_size)
    def init_hidden(self, batch_size):
        self.hidden = (torch.zeros(self.num_layers, batch_size, self.hidden_dim, device=device, requires_grad=True),
                       torch.zeros(self.num_layers, batch_size, self.hidden_dim, device=device, requires_grad=True))

    def forward(self, input):
        lstm_out, self.hidden = self.lstm(input.permute(1,0,2), self.hidden)
        return self.linear(lstm_out[-1])

In [7]:
predictions = {}
actuals = {}
models = {}

for station in station_cell_mapping['station'].unique():
    print(station)
    train_dataset, test_dataset, train_dataloader, val_dataloader, test_dataloader = get_datasets(station)
    onehot_vars = list(i for i,v in enumerate(train_dataset.fc_var_names) if v.startswith('month'))
    
    station_cell_mask = torch.zeros(train_dataset.x_conv.shape[-2:]).bool()
    for _, row in station_cell_mapping[station_cell_mapping['station'] == station][['row', 'col']].iterrows():
        station_cell_mask[row['col'], row['row']] = True
    
    best_loss_model = (-1, np.inf, None)
    model = LSTMRegression(train_dataset.x_conv.shape[2] * int(station_cell_mask.sum()) + len(onehot_vars), H, lstm_layers, batch_size).to(device)
    loss_fn = torch.nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    
    for epoch in range(epochs):
        model.train()
        train_losses = torch.tensor(0.0)
        for i, train_batch in enumerate(train_dataloader):
            x_train = train_batch['x_conv'][...,station_cell_mask].reshape(*train_batch['x_conv'].shape[:2], -1)
            x_train = torch.cat([x_train, train_batch['x_fc'][:,onehot_vars].unsqueeze(dim=1).repeat(1,x_train.shape[1],1)], dim=2).to(device)
            model.init_hidden(train_batch['x_conv'].shape[0])
            y_pred = model(x_train)
            
            train_loss = loss_fn(y_pred.reshape(-1), train_batch['y'].to(device))
            
            optimizer.zero_grad()
            train_loss.backward()
            optimizer.step()
            
            train_losses += train_loss.detach()
            
        train_loss = (train_losses / len(train_dataloader)).item()
        print('Epoch', epoch, 'mean loss:', train_loss)
        writer.add_scalar('loss_' + station, train_loss, epoch)
        
        model.eval()
        val_losses = torch.tensor(0.0)
        for i, val_batch in enumerate(val_dataloader):
            x_val = val_batch['x_conv'][...,station_cell_mask].reshape(*val_batch['x_conv'].shape[:2], -1)
            x_val = torch.cat([x_val, val_batch['x_fc'][:,onehot_vars].unsqueeze(dim=1).repeat(1,x_val.shape[1],1)], dim=2).to(device)
            model.init_hidden(val_batch['x_conv'].shape[0])
            y_pred = model(x_val)

            loss = loss_fn(y_pred.reshape(-1), val_batch['y'].to(device))
            val_losses += loss.detach()
            
        val_loss = (val_losses / len(val_dataloader)).item()
        print('Epoch', epoch, 'mean val loss:', val_loss)
        writer.add_scalar('loss_val_' + station, val_loss, epoch)
        if val_loss < best_loss_model[1] - min_improvement:
            best_loss_model = (epoch, val_loss, model.state_dict())  # new best model
            load_data.pickle_model('LSTM_VIC', model, station, time_stamp)
        elif epoch > best_loss_model[0] + patience:
            print('Patience exhausted in epoch {}. Best loss was {}'.format(epoch, best_loss_model[1]))
            break
        
    print('Using best model from epoch', str(best_loss_model[0]), 'which had loss', str(best_loss_model[1]))
    model.load_state_dict(best_loss_model[2])
    model.eval()
    predict = test_dataset.data_runoff.copy()
    predict['runoff'] = np.nan
    pred_array = np.array([])
    for i, test_batch in enumerate(test_dataloader):
        x_test = test_batch['x_conv'][...,station_cell_mask].reshape(*test_batch['x_conv'].shape[:2], -1)
        x_test = torch.cat([x_test, test_batch['x_fc'][:,onehot_vars].unsqueeze(dim=1).repeat(1,x_test.shape[1],1)], dim=2).to(device)
        model.init_hidden(test_batch['x_conv'].shape[0])
        pred_array = np.concatenate([pred_array, model(x_test).detach().cpu().numpy().reshape(-1)])
    predict['runoff'] = pred_array
    predictions[station] = predict.set_index('date')
    actuals[station] = test_dataset.data_runoff.set_index('date')['runoff']
    models[station] = model

02GA010
Epoch 0 mean loss: 269.8661193847656
Epoch 0 mean val loss: 259.4033508300781


/home/mgauch/miniconda3/envs/gwf/lib/python3.7/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type LSTMRegression. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Saved model as /home/mgauch/runoff-nn/src/../pickle/models/LSTM_VIC_02GA010_20190904-221524.pkl
Epoch 1 mean loss: 216.96676635742188
Epoch 1 mean val loss: 238.36561584472656
Saved model as /home/mgauch/runoff-nn/src/../pickle/models/LSTM_VIC_02GA010_20190904-221524.pkl
Epoch 2 mean loss: 207.55091857910156
Epoch 2 mean val loss: 251.96340942382812
Epoch 3 mean loss: 205.0540771484375
Epoch 3 mean val loss: 229.68760681152344
Saved model as /home/mgauch/runoff-nn/src/../pickle/models/LSTM_VIC_02GA010_20190904-221524.pkl
Epoch 4 mean loss: 203.03433227539062
Epoch 4 mean val loss: 228.04298400878906
Saved model as /home/mgauch/runoff-nn/src/../pickle/models/LSTM_VIC_02GA010_20190904-221524.pkl
Epoch 5 mean loss: 200.98953247070312
Epoch 5 mean val loss: 224.8544464111328
Saved model as /home/mgauch/runoff-nn/src/../pickle/models/LSTM_VIC_02GA010_20190904-221524.pkl
Epoch 6 mean loss: 200.76560974121094
Epoch 6 mean val loss: 224.59121704101562
Saved model as /home/mgauch/runoff-nn/src/

/home/mgauch/miniconda3/envs/gwf/lib/python3.7/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type LSTMRegression. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Epoch 1 mean loss: 129.02664184570312
Epoch 1 mean val loss: 43.57831573486328
Saved model as /home/mgauch/runoff-nn/src/../pickle/models/LSTM_VIC_02GA018_20190904-221524.pkl
Epoch 2 mean loss: 127.82717895507812
Epoch 2 mean val loss: 45.187198638916016
Epoch 3 mean loss: 127.62091827392578
Epoch 3 mean val loss: 44.85738754272461
Epoch 4 mean loss: 127.7883071899414
Epoch 4 mean val loss: 45.05954360961914
Epoch 5 mean loss: 127.288818359375
Epoch 5 mean val loss: 43.989967346191406
Epoch 6 mean loss: 124.99537658691406
Epoch 6 mean val loss: 43.088741302490234
Saved model as /home/mgauch/runoff-nn/src/../pickle/models/LSTM_VIC_02GA018_20190904-221524.pkl
Epoch 7 mean loss: 122.32376098632812
Epoch 7 mean val loss: 41.62773895263672
Saved model as /home/mgauch/runoff-nn/src/../pickle/models/LSTM_VIC_02GA018_20190904-221524.pkl
Epoch 8 mean loss: 122.03834533691406
Epoch 8 mean val loss: 39.8272590637207
Saved model as /home/mgauch/runoff-nn/src/../pickle/models/LSTM_VIC_02GA018_20190

...

Epoch 1 mean loss: 158.3089599609375
Epoch 1 mean val loss: 77.70523071289062
Saved model as /home/mgauch/runoff-nn/src/../pickle/models/LSTM_VIC_04214500_20190904-221524.pkl
Epoch 2 mean loss: 150.74594116210938
Epoch 2 mean val loss: 113.00439453125
Epoch 3 mean loss: 144.5663299560547
Epoch 3 mean val loss: 58.4184455871582
Saved model as /home/mgauch/runoff-nn/src/../pickle/models/LSTM_VIC_04214500_20190904-221524.pkl
Epoch 4 mean loss: 140.69342041015625
Epoch 4 mean val loss: 51.58137130737305
Saved model as /home/mgauch/runoff-nn/src/../pickle/models/LSTM_VIC_04214500_20190904-221524.pkl
Epoch 5 mean loss: 139.3389129638672
Epoch 5 mean val loss: 47.888336181640625
Saved model as /home/mgauch/runoff-nn/src/../pickle/models/LSTM_VIC_04214500_20190904-221524.pkl
Epoch 6 mean loss: 138.3080291748047
Epoch 6 mean val loss: 75.71868896484375
Epoch 7 mean loss: 137.40785217285156
Epoch 7 mean val loss: 80.00176239013672
Epoch 8 mean loss: 136.78968811035156
Epoch 8 mean val loss: 62.0

/home/mgauch/miniconda3/envs/gwf/lib/python3.7/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type LSTMRegression. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Epoch 1 mean loss: 70.1650161743164
Epoch 1 mean val loss: 94.75724792480469
Epoch 2 mean loss: 67.94573974609375
Epoch 2 mean val loss: 93.87583923339844
Saved model as /home/mgauch/runoff-nn/src/../pickle/models/LSTM_VIC_04215000_20190904-221524.pkl
Epoch 3 mean loss: 65.5283203125
Epoch 3 mean val loss: 98.34674835205078
Epoch 4 mean loss: 61.921974182128906
Epoch 4 mean val loss: 91.7748794555664
Saved model as /home/mgauch/runoff-nn/src/../pickle/models/LSTM_VIC_04215000_20190904-221524.pkl
Epoch 5 mean loss: 59.2553596496582
Epoch 5 mean val loss: 93.2301025390625
Epoch 6 mean loss: 56.17904281616211
Epoch 6 mean val loss: 90.57720184326172
Saved model as /home/mgauch/runoff-nn/src/../pickle/models/LSTM_VIC_04215000_20190904-221524.pkl
Epoch 7 mean loss: 54.530235290527344
Epoch 7 mean val loss: 92.50104522705078
Epoch 8 mean loss: 54.9930419921875
Epoch 8 mean val loss: 84.80831146240234
Saved model as /home/mgauch/runoff-nn/src/../pickle/models/LSTM_VIC_04215000_20190904-221524

/home/mgauch/miniconda3/envs/gwf/lib/python3.7/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type LSTMRegression. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Epoch 1 mean loss: 146.9365997314453
Epoch 1 mean val loss: 56.419776916503906
Epoch 2 mean loss: 143.8698272705078
Epoch 2 mean val loss: 49.35734939575195
Epoch 3 mean loss: 135.47230529785156
Epoch 3 mean val loss: 53.568424224853516
Epoch 4 mean loss: 130.01864624023438
Epoch 4 mean val loss: 44.41065216064453
Saved model as /home/mgauch/runoff-nn/src/../pickle/models/LSTM_VIC_04215500_20190904-221524.pkl
Epoch 5 mean loss: 128.7781219482422
Epoch 5 mean val loss: 44.281898498535156
Saved model as /home/mgauch/runoff-nn/src/../pickle/models/LSTM_VIC_04215500_20190904-221524.pkl
Epoch 6 mean loss: 129.10775756835938
Epoch 6 mean val loss: 47.72802734375
Epoch 7 mean loss: 123.00840759277344
Epoch 7 mean val loss: 43.25003433227539
Saved model as /home/mgauch/runoff-nn/src/../pickle/models/LSTM_VIC_04215500_20190904-221524.pkl
Epoch 8 mean loss: 122.88135528564453
Epoch 8 mean val loss: 48.2131462097168
Epoch 9 mean loss: 118.65462493896484
Epoch 9 mean val loss: 39.79816818237305
Sa

In [8]:
nse_list = []
for station, predict in predictions.items():
    nse = evaluate.evaluate_daily(station, predict['runoff'], actuals[station], writer=writer)
    nse_list.append(nse)
    
    print(station, '\tNSE: (clipped to 0)', nse_list[-1])

print('Median NSE (clipped to 0)', np.median(nse_list), '/ Min', np.min(nse_list), '/ Max', np.max(nse_list))

/home/mgauch/miniconda3/envs/gwf/lib/python3.7/site-packages/pandas/plotting/_converter.py:129: FutureWarning: Using an implicitly registered datetime converter for a matplotlib plotting method. The converter was registered by pandas on import. Future versions of pandas will require you to explicitly register matplotlib converters.

To register the converters:
	>>> from pandas.plotting import register_matplotlib_converters
	>>> register_matplotlib_converters()
  warnings.warn(msg, FutureWarning)


02GA010 	NSE: (clipped to 0) (-0.10419904867661822, 526.4528984327488)
02GA018 	NSE: (clipped to 0) (-0.11871320710109212, 280.96838003307425)
02GA038 	NSE: (clipped to 0) (0.0015788145982010837, 164.27868121118829)
02GA047 	NSE: (clipped to 0) (-0.3145064798727988, 103.04419537907975)
02GB001 	NSE: (clipped to 0) (-0.07393809374008353, 8065.141055013095)
02GB007 	NSE: (clipped to 0) (0.05355153485542474, 29.597350152688)
02GC002 	NSE: (clipped to 0) (0.19137659311065125, 103.91056165044317)
02GC007 	NSE: (clipped to 0) (0.004007740460455378, 30.00062594284005)
02GC010 	NSE: (clipped to 0) (0.2336474234256699, 45.55959921499974)
02GC018 	NSE: (clipped to 0) (0.25202727289514903, 50.65780058366654)
02GC026 	NSE: (clipped to 0) (0.04784143480626635, 153.1382836578956)
02GD004 	NSE: (clipped to 0) (0.010057082493312275, 54.50929954132059)
02GE007 	NSE: (clipped to 0) (0.30315371388624734, 25.208455151476944)
02GG002 	NSE: (clipped to 0) (0.23587331685953938, 184.29423318856044)
02GG003 	N

In [9]:
writer.close()

In [10]:
load_data.pickle_results('LSTM_VIC', (predictions, actuals), time_stamp)

'LSTM_VIC_20190904-221524.pkl'

In [11]:
datetime.now().strftime('%Y%m%d-%H%M%S')

'20190905-084243'